<a href="https://colab.research.google.com/github/Suliman8/FYP-1/blob/main/Final_Verifying_Missing_duplicate_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Handling Missing Values and Duplicates in UNSW-NB15 Dataset
# --------------------------------------------------------

# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
from google.colab import files
import io
import os

# Set display options for better output readability
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Upload dataset
print("Please upload your UNSW-NB15 training dataset CSV file...")
uploaded = files.upload()

# Get the name of the uploaded file
filename = list(uploaded.keys())[0]
print(f"Processing file: {filename}")

# Function to generate missing values report with fixed rounding issue
def generate_missing_values_report(df):
    """Generate an HTML report with details about missing values in the dataset."""
    # Calculate missing values statistics
    total_cells = df.size
    total_missing = df.isnull().sum().sum()

    # Create a DataFrame with missing values info - fixing the rounding issue
    missing_counts = df.isnull().sum()
    missing_percent = missing_counts / len(df) * 100

    missing_data = pd.DataFrame({
        'Column': df.columns,
        'Missing Values': missing_counts.values,
        'Percentage (%)': missing_percent.values.round(2)
    })

    # Sort by percentage of missing values
    missing_data = missing_data.sort_values('Percentage (%)', ascending=False).reset_index(drop=True)

    # Filter out columns with no missing values
    missing_data = missing_data[missing_data['Missing Values'] > 0]

    # Create HTML report
    html = f"""
    <html>
    <head>
        <title>Missing Values Report</title>
        <style>
            body {{ font-family: Arial, sans-serif; margin: 20px; }}
            h1, h2 {{ color: #2c3e50; }}
            .container {{ max-width: 1200px; margin: 0 auto; }}
            .summary {{ background-color: #f8f9fa; padding: 15px; border-radius: 5px; margin-bottom: 20px; }}
            table {{ border-collapse: collapse; width: 100%; margin-top: 20px; }}
            th, td {{ border: 1px solid #ddd; padding: 8px; text-align: left; }}
            th {{ background-color: #f2f2f2; }}
            tr:nth-child(even) {{ background-color: #f9f9f9; }}
            tr:hover {{ background-color: #f5f5f5; }}
            .warning {{ color: #e74c3c; }}
            .good {{ color: #27ae60; }}
        </style>
    </head>
    <body>
        <div class="container">
            <h1>Missing Values Report for UNSW-NB15 Dataset</h1>

            <div class="summary">
                <h2>Summary</h2>
                <p>Total cells in dataset: {total_cells:,}</p>
                <p>Total missing values: {total_missing:,}</p>
                <p>Percentage of missing values: {(total_missing/total_cells*100):.4f}%</p>
                <p>Number of columns with missing values: {len(missing_data)}</p>
                <p class="{'warning' if total_missing > 0 else 'good'}">
                    {'Dataset contains missing values that need to be addressed' if total_missing > 0 else 'Dataset has no missing values'}
                </p>
            </div>

            <h2>Columns with Missing Values</h2>
    """

    if len(missing_data) > 0:
        # Add table of columns with missing values
        html += """
            <table>
                <tr>
                    <th>Column</th>
                    <th>Missing Values</th>
                    <th>Percentage (%)</th>
                </tr>
        """

        for _, row in missing_data.iterrows():
            html += f"""
                <tr>
                    <td>{row['Column']}</td>
                    <td>{int(row['Missing Values']):,}</td>
                    <td>{row['Percentage (%)']:.2f}</td>
                </tr>
            """

        html += """
            </table>
        """
    else:
        html += "<p>No columns with missing values found.</p>"

    html += """
        </div>
    </body>
    </html>
    """

    # Display HTML in notebook
    display(HTML(html))

    return missing_data

# Step 1: Load the dataset
print("\nStep 1: Loading the UNSW-NB15 training dataset...")
train_df = pd.read_csv(io.BytesIO(uploaded[filename]))

# Display basic information about the dataset
print(f"\nDataset shape: {train_df.shape}")
print(f"Number of rows: {train_df.shape[0]}")
print(f"Number of columns: {train_df.shape[1]}")

# View first few rows
print("\nFirst few rows of the dataset:")
display(train_df.head())

# View data types
print("\nDataset column types:")
display(train_df.dtypes)

# Create a backup of the original dataset
train_original = train_df.copy()

# Step 2: Analyze missing values
print("\nStep 2: Analyzing missing values...")
# Check missing values
missing_values = train_df.isnull().sum()
missing_percent = (missing_values / len(train_df)) * 100
total_missing = missing_values.sum()
print(f"Total missing values: {total_missing}")

# Display columns with missing values
print("\nColumns with missing values:")
missing_info = pd.DataFrame({
    'Missing Values': missing_values[missing_values > 0],
    'Percentage (%)': missing_percent[missing_values > 0].round(2)
})
display(missing_info)

# Generate missing values report (with fixed rounding)
print("\nGenerating detailed missing values report...")
missing_data = generate_missing_values_report(train_df)

# Visualize missing values only if there are any
if total_missing > 0:
    plt.figure(figsize=(15, 8))
    sns.heatmap(train_df.isnull(), cbar=False, cmap='viridis', yticklabels=False)
    plt.title('Missing Values in UNSW-NB15 Training Dataset')
    plt.tight_layout()
    plt.show()
else:
    print("No missing values to visualize.")

# Step 3: Check for duplicate rows
print("\nStep 3: Analyzing duplicate rows...")
duplicate_count = train_df.duplicated().sum()
duplicate_percentage = round((duplicate_count / len(train_df) * 100), 2)  # Fixed: using round() function
print(f"Number of duplicate rows: {duplicate_count} ({duplicate_percentage}%)")

# If there are duplicates, show a few examples
if duplicate_count > 0:
    print("\nSome examples of duplicate rows:")
    duplicated_rows = train_df[train_df.duplicated(keep='first')]
    display(duplicated_rows.head(3))

# Step 4: Handle missing values
print("\nStep 4: Handling missing values...")
if total_missing > 0:
    # For categorical columns
    cat_columns = ['proto', 'service', 'state', 'attack_cat']
    for col in cat_columns:
        if col in train_df.columns and train_df[col].isnull().sum() > 0:
            # Fill missing categorical values with the most frequent value
            most_frequent = train_df[col].mode()[0]
            train_df[col] = train_df[col].fillna(most_frequent)
            print(f"  - Filled missing values in '{col}' with mode: {most_frequent}")

    # For numerical columns
    num_columns = train_df.select_dtypes(include=['number']).columns
    for col in num_columns:
        if train_df[col].isnull().sum() > 0:
            # Fill missing numerical values with median (more robust than mean)
            median_val = train_df[col].median()
            train_df[col] = train_df[col].fillna(median_val)
            print(f"  - Filled missing values in '{col}' with median: {median_val}")

    # Check if any missing values remain
    remaining_missing = train_df.isnull().sum().sum()
    if remaining_missing > 0:
        print(f"\nWarning: {remaining_missing} missing values still remain in the dataset!")
    else:
        print("\nAll missing values have been handled successfully!")
else:
    print("No missing values to handle in the dataset.")

# Step 5: Remove duplicate rows if needed
print("\nStep 5: Removing duplicate rows...")
if duplicate_count > 0:
    train_df_no_duplicates = train_df.drop_duplicates()
    removed_duplicates = len(train_df) - len(train_df_no_duplicates)
    print(f"Removed {removed_duplicates} duplicate rows")

    # Show shape after removing duplicates
    print(f"Dataset shape after removing duplicates: {train_df_no_duplicates.shape}")
else:
    print("No duplicate rows to remove.")
    train_df_no_duplicates = train_df.copy()

# Step 6: Save the cleaned dataset
print("\nStep 6: Preparing to download the cleaned dataset...")
# Save the cleaned dataset
cleaned_filename = 'UNSW_NB15_training_cleaned.csv'
train_df_no_duplicates.to_csv(cleaned_filename, index=False)
print(f"Cleaned dataset saved as '{cleaned_filename}'")

# Allow user to download the cleaned dataset
files.download(cleaned_filename)

# Step 7: Generate summary report
print("\nStep 7: Generating summary report...")
# Create a summary of the preprocessing steps
data_reduction = len(train_original) - len(train_df_no_duplicates)
data_reduction_percent = round((data_reduction / len(train_original) * 100), 2)  # Fixed: using round() function

# Check if any preprocessing was actually done
if total_missing == 0 and duplicate_count == 0:
    preprocessing_message = "No preprocessing was required. The dataset had no missing values or duplicates."
else:
    preprocessing_message = ""
    if total_missing > 0:
        preprocessing_message += "- Filled missing values in categorical columns with mode\n   - Filled missing values in numerical columns with median\n   "
    if duplicate_count > 0:
        preprocessing_message += "- Removed duplicate rows"

# Create the summary report
summary_report = f"""
UNSW-NB15 Dataset Preprocessing Summary
======================================

1. Original Dataset Information
   - Number of rows: {train_original.shape[0]}
   - Number of columns: {train_original.shape[1]}

2. Missing Values
   - Total missing values: {total_missing}
   - Percentage of missing values: {round(total_missing/(train_original.shape[0]*train_original.shape[1])*100, 4)}%
   - Number of columns with missing values: {len(missing_info)}

3. Duplicate Rows
   - Number of duplicate rows: {duplicate_count}
   - Percentage of duplicate rows: {duplicate_percentage}%

4. Preprocessing Steps Applied
   {preprocessing_message}

5. Cleaned Dataset Information
   - Number of rows after cleaning: {train_df_no_duplicates.shape[0]}
   - Number of columns: {train_df_no_duplicates.shape[1]}
   - Total rows removed: {data_reduction}
   - Percentage of data reduction: {data_reduction_percent}%

6. Next Steps
   - Encode categorical features
   - Normalize/standardize numerical features
   - Perform feature engineering
   - Address class imbalance if necessary
"""

print(summary_report)

# Save summary report to file
summary_filename = 'preprocessing_summary.txt'
with open(summary_filename, 'w') as f:
    f.write(summary_report)

# Allow user to download the summary report
files.download(summary_filename)

print("\nPreprocessing completed successfully!")
print("You can now proceed to the next preprocessing steps: encoding categorical features, normalization, and feature engineering.")

Please upload your UNSW-NB15 training dataset CSV file...


Saving UNSW_NB15_training-set.csv to UNSW_NB15_training-set (1).csv
Processing file: UNSW_NB15_training-set (1).csv

Step 1: Loading the UNSW-NB15 training dataset...

Dataset shape: (175341, 45)
Number of rows: 175341
Number of columns: 45

First few rows of the dataset:


,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,dloss,sinpkt,dinpkt,sjit,djit,swin,stcpb,dtcpb,dwin,tcprtt,synack,ackdat,smean,dmean,trans_depth,response_body_len,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.121478,tcp,-,FIN,6,4,258,172,74.087490,252,254,14158.942380,8495.365234,0,0,24.295600,8.375000,30.177547,11.830604,255,621772692,2202533631,255,0.000000,0.000000,0.000000,43,43,0,0,1,0,1,1,1,1,0,0,0,1,1,0,Normal,0
1,2,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,62,252,8395.112305,503571.312500,2,17,49.915000,15.432865,61.426934,1387.778330,255,1417884146,3077387971,255,0.000000,0.000000,0.000000,52,1106,0,0,43,1,1,1,1,2,0,0,0,1,6,0,Normal,0
2,3,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,62,252,1572.271851,60929.230470,1,6,231.875571,102.737203,17179.586860,11420.926230,255,2116150707,2963114973,255,0.111897,0.061458,0.050439,46,824,0,0,7,1,2,1,1,3,0,0,0,2,6,0,Normal,0
3,4,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,62,252,2740.178955,3358.622070,1,3,152.876547,90.235726,259.080172,4991.784669,255,1107119177,1047442890,255,0.000000,0.000000,0.000000,52,64,0,0,1,1,2,1,1,3,1,1,0,2,1,0,Normal,0
4,5,0.449454,tcp,-,FIN,10,6,534,268,33.373826,254,252,8561.499023,3987.059814,2,1,47.750333,75.659602,2415.837634,115.807000,255,2436137549,1977154190,255,0.128381,0.071147,0.057234,53,45,0,0,43,1,2,2,1,40,0,0,0,2,39,0,Normal,0



Dataset column types:


,0
id,int64
dur,float64
proto,object
service,object
state,object
spkts,int64
dpkts,int64
sbytes,int64
dbytes,int64
rate,float64


...

<IPython.core.display.Javascript object>


Step 7: Generating summary report...

UNSW-NB15 Dataset Preprocessing Summary

1. Original Dataset Information
   - Number of rows: 175341
   - Number of columns: 45

2. Missing Values
   - Total missing values: 0
   - Percentage of missing values: 0.0%
   - Number of columns with missing values: 0

3. Duplicate Rows
   - Number of duplicate rows: 0
   - Percentage of duplicate rows: 0.0%

4. Preprocessing Steps Applied
   No preprocessing was required. The dataset had no missing values or duplicates.

5. Cleaned Dataset Information
   - Number of rows after cleaning: 175341
   - Number of columns: 45
   - Total rows removed: 0
   - Percentage of data reduction: 0.0%

6. Next Steps
   - Encode categorical features
   - Normalize/standardize numerical features
   - Perform feature engineering
   - Address class imbalance if necessary



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Preprocessing completed successfully!
You can now proceed to the next preprocessing steps: encoding categorical features, normalization, and feature engineering.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
